In [1]:
import pandas as pd
def load_data_set():
    """
    Load a sample data set (From Data Mining: Concepts and Techniques, 3th Edition)
    Returns:
        A data set: A list of transactions. Each transaction contains several items.
    """
    df = pd.read_excel('老年人比重及影响因素.xlsx')
    df = df.drop(['年份'], axis=1)
    df = df.fillna(0)
    new_data = df.values
    return new_data

In [2]:
def create_C1(data_set):
    """
    Create frequent candidate 1-itemset C1 by scaning data set.
    Args:
        data_set: A list of transactions. Each transaction contains several items.
    Returns:
        C1: A set which contains all frequent candidate 1-itemsets
    """
    C1 = set()
    for t in data_set:
        for item in t:
            item_set = frozenset([item])
            C1.add(item_set)
    return C1

In [3]:
def is_apriori(Ck_item, Lksub1):
    """
    Judge whether a frequent candidate k-itemset satisfy Apriori property.
    Args:
        Ck_item: a frequent candidate k-itemset in Ck which contains all frequent
                 candidate k-itemsets.
        Lksub1: Lk-1, a set which contains all frequent candidate (k-1)-itemsets.
    Returns:
        True: satisfying Apriori property.
        False: Not satisfying Apriori property.
    """
    for item in Ck_item:
        sub_Ck = Ck_item - frozenset([item])
        if sub_Ck not in Lksub1:
            return False
    return True


In [4]:
def create_Ck(Lksub1, k):
    """
    Create Ck, a set which contains all all frequent candidate k-itemsets
    by Lk-1's own connection operation.
    Args:
        Lksub1: Lk-1, a set which contains all frequent candidate (k-1)-itemsets.
        k: the item number of a frequent itemset.
    Return:
        Ck: a set which contains all all frequent candidate k-itemsets.
    """
    Ck = set()
    len_Lksub1 = len(Lksub1)
    list_Lksub1 = list(Lksub1)
    for i in range(len_Lksub1):
        for j in range(1, len_Lksub1):
            l1 = list(list_Lksub1[i])
            l2 = list(list_Lksub1[j])
            l1.sort()
            l2.sort()
            if l1[0:k-2] == l2[0:k-2]:
                Ck_item = list_Lksub1[i] | list_Lksub1[j]
                # pruning
                if is_apriori(Ck_item, Lksub1):
                    Ck.add(Ck_item)
    return Ck

In [5]:
def generate_Lk_by_Ck(data_set, Ck, min_support, support_data):
    """
    Generate Lk by executing a delete policy from Ck.
    Args:
        data_set: A list of transactions. Each transaction contains several items.
        Ck: A set which contains all all frequent candidate k-itemsets.
        min_support: The minimum support.
        support_data: A dictionary. The key is frequent itemset and the value is support.
    Returns:
        Lk: A set which contains all all frequent k-itemsets.
    """
    Lk = set()
    item_count = {}
    for t in data_set:
        for item in Ck:
            if item.issubset(t):
                if item not in item_count:
                    item_count[item] = 1
                else:
                    item_count[item] += 1
    t_num = float(len(data_set))
    for item in item_count:
        if (item_count[item] / t_num) >= min_support:
            Lk.add(item)
            support_data[item] = item_count[item] / t_num
    return Lk

In [6]:
def generate_L(data_set, k, min_support):
    """
    Generate all frequent itemsets.
    Args:
        data_set: A list of transactions. Each transaction contains several items.
        k: Maximum number of items for all frequent itemsets.
        min_support: The minimum support.
    Returns:
        L: The list of Lk.
        support_data: A dictionary. The key is frequent itemset and the value is support.
    """
    support_data = {}
    C1 = create_C1(data_set)
    L1 = generate_Lk_by_Ck(data_set, C1, min_support, support_data)
    Lksub1 = L1.copy()
    L = []
    L.append(Lksub1)
    for i in range(2, k+1):
        Ci = create_Ck(Lksub1, i)
        Li = generate_Lk_by_Ck(data_set, Ci, min_support, support_data)
        Lksub1 = Li.copy()
        L.append(Lksub1)
    return L, support_data

In [7]:
def generate_big_rules(L, support_data, min_conf):
    """
    Generate big rules from frequent itemsets.
    Args:
        L: The list of Lk.
        support_data: A dictionary. The key is frequent itemset and the value is support.
        min_conf: Minimal confidence.
    Returns:
        big_rule_list: A list which contains all big rules. Each big rule is represented
                       as a 3-tuple.
    """
    big_rule_list = []
    sub_set_list = []
    for i in range(0, len(L)):
        for freq_set in L[i]:
            for sub_set in sub_set_list:
                if sub_set.issubset(freq_set):
                    conf = support_data[freq_set] / support_data[freq_set - sub_set]
                    big_rule = (freq_set - sub_set, sub_set, conf)
                    if conf >= min_conf and big_rule not in big_rule_list:
                        # print freq_set-sub_set, " => ", sub_set, "conf: ", conf
                        big_rule_list.append(big_rule)
            sub_set_list.append(freq_set)
    return big_rule_list

In [10]:
"""
    Test
    """
data_set = load_data_set()
L, support_data = generate_L(data_set, k=3, min_support=0.1)
big_rules_list = generate_big_rules(L, support_data, min_conf=1)
L

[{frozenset({'A2'}),
  frozenset({'G2'}),
  frozenset({'G1'}),
  frozenset({'H2'}),
  frozenset({'D2'}),
  frozenset({'C3'}),
  frozenset({'D1'}),
  frozenset({'E1'}),
  frozenset({'F1'}),
  frozenset({'H3'}),
  frozenset({'C1'}),
  frozenset({'F2'}),
  frozenset({'H1'}),
  frozenset({'B4'}),
  frozenset({'A1'}),
  frozenset({'C2'}),
  frozenset({'F3'}),
  frozenset({'G3'}),
  frozenset({'B3'}),
  frozenset({'B2'}),
  frozenset({'E2'}),
  frozenset({'D3'})},
 {frozenset({'A2', 'G3'}),
  frozenset({'D1', 'F2'}),
  frozenset({'B3', 'G3'}),
  frozenset({'B2', 'E1'}),
  frozenset({'A1', 'H1'}),
  frozenset({'C2', 'F2'}),
  frozenset({'D2', 'F2'}),
  frozenset({'E2', 'G3'}),
  frozenset({'G3', 'H3'}),
  frozenset({'A2', 'B2'}),
  frozenset({'A1', 'E1'}),
  frozenset({'C2', 'D1'}),
  frozenset({'D3', 'G3'}),
  frozenset({'B3', 'D2'}),
  frozenset({'A2', 'D1'}),
  frozenset({'C3', 'F3'}),
  frozenset({'B2', 'G1'}),
  frozenset({'E2', 'F1'}),
  frozenset({'C1', 'E1'}),
  frozenset({'D1', 'E1'}

In [11]:
def get_support(x):
    for Lk in L:
        # print("="*50)
        # print("frequent " + str(len(list(Lk)[0])) + "-itemsets\t\tsupport")
        # print("="*50)
        for freq_set in Lk:
            if x == freq_set:
                return support_data[freq_set]
        # print(freq_set, support_data[freq_set])

print("Big Rules")
for item in big_rules_list:
    tiaojian = str(item[1])
    if 'A' in tiaojian and ',' not in tiaojian:
        support = get_support(item[0])
        print(item[0], "=>", item[1], "conf: ", item[2],"support: ", support)

Big Rules
frozenset({'E2'}) => frozenset({'A2'}) conf:  1.0 support:  0.3333333333333333
frozenset({'B4'}) => frozenset({'A2'}) conf:  1.0 support:  0.14285714285714285
frozenset({'D2', 'G3'}) => frozenset({'A2'}) conf:  1.0 support:  0.14285714285714285
frozenset({'B4', 'G3'}) => frozenset({'A2'}) conf:  1.0 support:  0.14285714285714285
frozenset({'E2', 'F1'}) => frozenset({'A2'}) conf:  1.0 support:  0.14285714285714285
frozenset({'F1', 'G3'}) => frozenset({'A2'}) conf:  1.0 support:  0.14285714285714285
frozenset({'E2', 'B4'}) => frozenset({'A2'}) conf:  1.0 support:  0.14285714285714285
frozenset({'B3', 'G3'}) => frozenset({'A2'}) conf:  1.0 support:  0.14285714285714285
frozenset({'E2', 'D3'}) => frozenset({'A2'}) conf:  1.0 support:  0.14285714285714285
frozenset({'C1', 'H1'}) => frozenset({'A1'}) conf:  1.0 support:  0.14285714285714285
frozenset({'E2', 'C3'}) => frozenset({'A2'}) conf:  1.0 support:  0.23809523809523808
frozenset({'D1', 'G2'}) => frozenset({'A2'}) conf:  1.0 s